In [ ]:
# best 3 solutions based on EXP1 & 3 in turns of f1 weighted is:
# XGBcls, RoadNet+Segment TOTAL, 0.5284741533025521; + smote_svm: 0.5396583161
# XGBreg, RoadNet+Segment NO_TOTAL, 0.512063968; type instead of total + smote_svm: 0.533828674; + smote_svm+mrmr: 0.5330293468
# GDBcls, RoadNet+Segment TOTAL, 0.5107103184755486; +smote_svm + rfecv_linsvm: 0.5178884611

In [ ]:
from src.experiment_based_function import SEEDS
import pandas as pd
%matplotlib inline
import os
import glob

In [ ]:
def get_exp5_result(eval_or_grid):
    res = []
    for directoy in glob.glob('data/exp5/*'):
        qa = os.path.basename(directoy)
        q, a = qa.split('#')
        for seed in SEEDS:
            df = pd.read_csv('%s/seed_%d/%s_res.csv' % (directoy, seed, eval_or_grid), index_col=0)
            df['q'] = q
            df['a'] = a
            df['seed'] = seed
            res.append(df)
    return pd.concat(res)

In [ ]:
def get_exp5_result(eval_or_grid='eval'):
    res = []
    for directoy in glob.glob('data/exp5/*/*/*'):
#         print(directoy)
        df = pd.read_csv('%s/%s_res.csv' % (directoy, eval_or_grid), index_col=0)
        frac, sample_seed, train_test_seed = directoy.replace('data/exp5/', '').split('/')
        df['frac'] = int(frac[5:])
        df['sample_seed'] = int(sample_seed[7:])
        df['train_test_seed'] = int(train_test_seed[5:])
        res.append(df)
    return pd.concat(res, ignore_index=0)

In [ ]:
eval_res = get_exp5_result()

In [ ]:
eval_res.columns

In [ ]:
print('train', eval_res.train_n_classes.value_counts().to_dict())
print('test', eval_res.test_n_classes.value_counts().to_dict())

print('train', 'test')
print(eval_res.apply(lambda x: (x.train_n_classes, x.test_n_classes), axis=1).value_counts())

In [ ]:
f1_choices = ['test_f1_weighted', 'test_f1_macro', 'test_f1_micro']

In [ ]:
keys = ['frac', 'upsample', 'ftr_combo_name', 'model_name', 'feature_selection']

In [ ]:
info = eval_res.groupby(['frac','upsample']).mean()[['#test_sample', '#train_sample','test_n_classes','train_n_classes', ]]

In [ ]:
means = eval_res.groupby(keys+['sample_seed']).mean()[f1_choices].groupby(keys).mean()
stds = eval_res.groupby(keys+['sample_seed']).std()[f1_choices].groupby(keys).std()
means = means.reset_index().merge(info.reset_index(), how='left').set_index(keys)

In [ ]:
def get_paper_table(means, f1_type, index_cols, column_col='ftr_combo_name'):
    cols_name_convert = {'RoadNet': '1RoadNet', 'RoadNet+Segment_NO_TOTAL_~2014': '5RoadNet+Segment type',
     'RoadNet+Segment_TOTAL_~2014': '4RoadNet+Segment total', 'Segment_NO_TOTAL_~2014': '3Segment type',
     'Segment_TOTAL_~2014': '2Segment total'}
    means_for_pivot = means.reset_index()
    means_for_pivot['ftr_combo_name'] = means_for_pivot['ftr_combo_name'].apply(lambda x: cols_name_convert[x])
    means_for_pivot['key'] = means_for_pivot.apply(lambda x: tuple([x[col] for col in index_cols]), axis=1)
    table = means_for_pivot.reset_index().pivot(index='key', columns=column_col, values=f1_type)
    table.index = pd.MultiIndex.from_tuples(table.index, names=index_cols)    
    return table

In [ ]:
index_cols = [k for k in keys if k!='frac']
# ['test_n_classes','train_n_classes']+
table = get_paper_table(means, f1_type, index_cols, column_col='frac')

In [ ]:
table